In [55]:
import os
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import datetime

In [59]:
# Functions

def read_page(driver, url):
    driver.get(url)
    text = driver.page_source
    return text


def get_league_matches(driver, country, league, league_url):
    print "Country:", country
    print "League:", league
    print "League URL:", league_url
    print

    league_matches = []

    # Read url
    try:
        headers = {'User-agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) \
                    Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1'}
        r = requests.get(league_url, headers=headers)
        text = r.text
    except:
        pass

    if text:
        soup = BeautifulSoup(text, "lxml")
        seasons_details = []
        for i in soup.findAll("div", {"class": "main-menu2 main-menu-gray"})[0].findAll("li"):
            season_url = "".join([site_url, i.span.a["href"]])
            season = int(i.text.split("/")[-2])
            seasons_details.append([season, season_url])            
        seasons_details = sorted(seasons_details)   

        seasons_details = seasons_details[-1:]
        for each_season in seasons_details:
            season, seasons_url = each_season
            print "Season:", season
            print "Season URL:", seasons_url
            season_matches = get_season_matches(driver, season, country, league, seasons_url)
            if season_matches:
                league_matches.extend(season_matches)
                print
        return league_matches
    else:
        return None


def get_season_matches(driver, season, country, league, seasons_url): 
    season_matches = []
    try:
        text = read_page(driver, seasons_url)
    except Exception, e:
            print "Season matches error:", e
            print ">>", seasons_url
    print "Read season url:", len(text)
    if text:
            try:
                page_numbers = list(set(re.findall('href="#/page/([0-9]{1})/"', text)))
            except Exception, e:
                page_numbers = ""
            if page_numbers:
                max_page_num = max(map(int, page_numbers))
                pages_urls = map(lambda x: "".join([seasons_url, "#/page/", str(x), "/"]), range(1, max_page_num+2)) # max_page_num+1))

                for pages_url in pages_urls:
                    page_matches = get_page_matches(driver, season, country, league, pages_url)
                    season_matches.extend(page_matches)
                return season_matches
            else:
                return None
    else:
        return None


def get_page_matches(driver, season, country, league, pages_url):
    page_matches = []
    try:
        text = read_page(driver, pages_url)
    except Exception, e:
        print "Get page matches error:", e
    if text:
        try:
            odds_type = re.findall('id="user-header-oddsformat-expander"><span>(.*?)</span>', text)
            if odds_type:
                odds_type = odds_type[0]
            else:
                odds_type = ""
        except Exception, e:
            odds_type = ""

        soup = BeautifulSoup(text)
        for i in soup.findAll("tr", {"class":re.compile("deactivate")}):
            try:
                timestamp = int(re.findall("[0-9]{10}", str(i.get_text))[0])
                datetime_date = datetime.datetime.utcfromtimestamp(timestamp)
                match_date = datetime_date.strftime('%d-%m-%Y')
                match_time = datetime_date.strftime('%H:%M')
            except Exception, e:
                timestamp, datetime_date, match_date, match_time = "", "", "", ""     

            try:
                ftsc = i.findAll("td", {"class": re.compile("table-score")})[0].text
                fthg, ftag = map(int, ftsc.split(":"))
                ftr_dict = {1: "H", 0: "D", -1: "A"}
                ftr = ftr_dict[cmp(fthg, ftag)]
            except Exception, e:
                ftsc, fthg, ftag, ftr = "", "", "", ""

            try:
                avoh, avod, avoa = re.findall('odds_text">(.*?)</a>', str(i.get_text))
            except Exception, e:
                avoh, avod, avoa = "", "", ""

            try:
                match_url = i.findAll("td", {"class": "name table-participant"})[0]
                hometeam, awayteam = map(lambda x: x.strip(), match_url.text.split("-"))
                match_url = "".join([site_url, match_url.a["href"]])
            except Exception, e:
                match_url, hometeam, awayteam = "", "", ""

            print ">>", match_date, match_time, hometeam, awayteam, ftsc, fthg, ftag, avoh, avod, avoa

            match_details = {"SEASON": season, "COUNTRY": country, "LEAGUE": league, "DATE": match_date, 
                             "TIME": match_time, "HOMETEAM": hometeam, "AWAYTEAM": awayteam, 
                             "FTSC": ftsc, "FTHG": fthg, "FTAG": ftag, "FTR": ftr, "AVOH": avoh, "AVOD": avod, 
                             "AVOA": avoa, "MATCH_URL": match_url, "TIMESTAMP": timestamp, "ODDS_TYPE": odds_type,
                             "PAGE_URL": pages_url}
            page_matches.append(match_details)
        return page_matches
    else:
        return None, None


In [62]:
## unchanged from http://stackoverflow.com/questions/30745587/why-does-selenium-return-the-source-of-the-previously-loaded-page-in-python 

def get_league_matches(driver, country, league, league_url):
    print "Country:", country
    print "League:", league
    print "League URL:", league_url
    print

    league_matches = []

    # Read url
    try:
        headers = {'User-agent': 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) \
                    Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1'}
        r = requests.get(league_url, headers=headers)
        text = r.text
    except:
        pass

    if text:
        soup = BeautifulSoup(text)
        seasons_details = []
        for i in soup.findAll("div", {"class": "main-menu2 main-menu-gray"})[0].findAll("li"):
            season_url = "".join([site_url, i.span.a["href"]])
            season = int(i.text.split("/")[-2])
            seasons_details.append([season, season_url])            
        seasons_details = sorted(seasons_details)   

        seasons_details = seasons_details[-1:]
        for each_season in seasons_details:
            season, seasons_url = each_season
            print "Season:", season
            print "Season URL:", seasons_url
            season_matches = get_season_matches(driver, season, country, league, seasons_url)
            if season_matches:
                league_matches.extend(season_matches)
                print
        return league_matches
    else:
        return None


def get_season_matches(driver, season, country, league, seasons_url):
    season_matches = []
    try:
        text = read_page(driver, seasons_url)
    except Exception, e:
            print "Season matches error:", e
            print ">>", seasons_url
    print "Read season url:", len(text)
    if text:
            try:
                page_numbers = list(set(re.findall('href="#/page/([0-9]{1})/"', text)))
            except Exception, e:
                page_numbers = ""
            if page_numbers:
                max_page_num = max(map(int, page_numbers))
                pages_urls = map(lambda x: "".join([seasons_url, "#/page/", str(x), "/"]), range(1, max_page_num+1)) # max_page_num+2))

                for pages_url in pages_urls:
                    page_matches = get_page_matches(driver, season, country, league, pages_url)
                    season_matches.extend(page_matches)
                return season_matches
            else:
                return None
    else:
        return None


def get_page_matches(driver, season, country, league, pages_url):
    page_matches = []
    try:
        text = read_page(driver, pages_url)
    except Exception, e:
        print "Get page matches error:", e
    if text:
        try:
            odds_type = re.findall('id="user-header-oddsformat-expander"><span>(.*?)</span>', text)
            if odds_type:
                odds_type = odds_type[0]
            else:
                odds_type = ""
        except Exception, e:
            odds_type = ""

        soup = BeautifulSoup(text)
        for i in soup.findAll("tr", {"class":re.compile("deactivate")}):
            try:
                timestamp = int(re.findall("[0-9]{10}", str(i.get_text))[0])
                datetime_date = datetime.datetime.utcfromtimestamp(timestamp)
                match_date = datetime_date.strftime('%d-%m-%Y')
                match_time = datetime_date.strftime('%H:%M')
            except Exception, e:
                timestamp, datetime_date, match_date, match_time = "", "", "", ""     

            try:
                ftsc = i.findAll("td", {"class": re.compile("table-score")})[0].text
                fthg, ftag = map(int, ftsc.split(":"))
                ftr_dict = {1: "H", 0: "D", -1: "A"}
                ftr = ftr_dict[cmp(fthg, ftag)]
            except Exception, e:
                ftsc, fthg, ftag, ftr = "", "", "", ""

            try:
                avoh, avod, avoa = re.findall('odds_text">(.*?)</a>', str(i.get_text))
            except Exception, e:
                avoh, avod, avoa = "", "", ""

            try:
                match_url = i.findAll("td", {"class": "name table-participant"})[0]
                hometeam, awayteam = map(lambda x: x.strip(), match_url.text.split("-"))
                match_url = "".join([site_url, match_url.a["href"]])
            except Exception, e:
                match_url, hometeam, awayteam = "", "", ""

            print ">>", match_date, match_time, hometeam, awayteam, ftsc, fthg, ftag, avoh, avod, avoa

            match_details = {"SEASON": season, "COUNTRY": country, "LEAGUE": league, "DATE": match_date, 
                             "TIME": match_time, "HOMETEAM": hometeam, "AWAYTEAM": awayteam, 
                             "FTSC": ftsc, "FTHG": fthg, "FTAG": ftag, "FTR": ftr, "AVOH": avoh, "AVOD": avod, 
                             "AVOA": avoa, "MATCH_URL": match_url, "TIMESTAMP": timestamp, "ODDS_TYPE": odds_type,
                             "PAGE_URL": pages_url}
            page_matches.append(match_details)
        return page_matches
    else:
        return None, None

In [63]:
# Script
site_url = "http://www.oddsportal.com"
soccer_url = "http://www.oddsportal.com/results/#soccer" 

driver = webdriver.PhantomJS()
country = "Germany"
league = "Bundesliga"
league_url = "http://www.oddsportal.com/soccer/germany/bundesliga/results/"
league_matches = get_league_matches(driver, country, league, league_url)

Country: Germany
League: Bundesliga
League URL: http://www.oddsportal.com/soccer/germany/bundesliga/results/

Season: 2015
Season URL: http://www.oddsportal.com/soccer/germany/bundesliga/results/
Read season url: 195717


In [64]:
#match_odds = {k:v for k,v in (x.split(':') for x in league_matches) }

fields = []
for match in league_matches:
    for item in match:
        if item not in fields:
            fields.append(item)
    
print fields

[]


In [50]:
list = []
dict = {}
for match in league_matches:
    for field in match:
        dict[field] = match[field]
    list.append(dict)
print list

[{'LEAGUE': 'Bundesliga', 'FTR': 'H', 'FTSC': u'5:0', 'MATCH_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/bayern-munich-hamburger-pzmLNQWJ/', 'SEASON': 2015, 'HOMETEAM': u'Bayern Munich', 'TIMESTAMP': 1439577000, 'FTHG': 5, 'AVOD': '+989', 'AWAYTEAM': u'Hamburger SV', 'AVOA': '+2728', 'TIME': '18:30', 'DATE': '14-08-2015', 'ODDS_TYPE': u'US Odds', 'FTAG': 0, 'COUNTRY': 'Germany', 'AVOH': '-1111', 'PAGE_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/'}, {'LEAGUE': 'Bundesliga', 'FTR': 'H', 'FTSC': u'5:0', 'MATCH_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/bayern-munich-hamburger-pzmLNQWJ/', 'SEASON': 2015, 'HOMETEAM': u'Bayern Munich', 'TIMESTAMP': 1439577000, 'FTHG': 5, 'AVOD': '+989', 'AWAYTEAM': u'Hamburger SV', 'AVOA': '+2728', 'TIME': '18:30', 'DATE': '14-08-2015', 'ODDS_TYPE': u'US Odds', 'FTAG': 0, 'COUNTRY': 'Germany', 'AVOH': '-1111', 'PAGE_URL': 'http://www.oddsportal.com/soccer/germany/bundesliga/results/#/page/5/'}, {

In [54]:
keys = league_matches[0].keys()
with open('../data/match_odds.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(league_matches)

In [ ]:
        row = []
        for field in fields:
            if field in item:
                row.append(item[field])
            else:
                row.append("")
        print row

In [36]:
with open('../data/match_odds.csv', 'wb') as f:
    w = csv.writer(f, quoting = csv.QUOTE_ALL)
    w.writerow(league_matches)

In [25]:
## adapted rom https://gist.github.com/tjvc/12e393c48e0025509021

from collections import OrderedDict
import csv
import json
import sys

infile = "data/allfinal.json"
outfile = open("data/playerdata.csv", "w")

writer = csv.writer(outfile, delimiter=",")

data = json.load(open(infile), object_pairs_hook=OrderedDict)

# Recursively flatten JSON
def flatten(structure, key="", path="", flattened=None): 
    if flattened is None:
        flattened = OrderedDict()    
    if type(structure) not in(OrderedDict, list):
        flattened[((path + "_") if path else "") + key] = structure    
    elif isinstance(structure, list):
        for i, item in enumerate(structure):
            flatten(item, "", path + "_" + key, flattened)    
    else:
        for new_key, value in structure.items():
            flatten(value, new_key, path + "_" + key, flattened)    
    return flattened

# Write fields
fields = []
for result in data:
    flattened = flatten(result)
    for k, v in flattened.iteritems():
        if k not in fields:
            fields.append(k)
writer.writerow(fields)
print "Flattened JSON to the following fields:"
print fields

# Write values
for result in data:
    flattened = flatten(result)
    row = []
    for field in fields:
        if field in flattened.iterkeys():
            row.append(flattened[field])
        else:
            row.append("")
    writer.writerow(row)
print "Stored CSV as " + outfile.name

IOError: [Errno 2] No such file or directory: 'data/playerdata.csv'